In [1]:
def max_codage_lz78(code):
    T = [i[0] for i in code]
    return max(T)
def I_zigzag(I):
    l,c=I.shape  
    li=[]
    T=np.fliplr(I)
    for i in range(c-1,-l,-1):
        A=np.diagonal(T,i)
        if(abs(i)%2==(c-1)%2):
            li.extend(reversed(A))
        else:
            li.extend(A)
    return li

def lz78c(sequence) :
    sequence = list(sequence)
    dictt = {"":0}
    etiquette = []
    courant = ""
    for car in sequence :
        courant += car
        if courant not in dictt.keys() :
            dictt[courant] = len(dictt.keys())
            etiquette.append((dictt[courant[:-1]],car))
            courant = ""
            
    if courant != "" :                  
        etiquette.append((dictt[courant],""))
    return etiquette



def to_parts(image_array,m):
    l, c = image_array.shape
    nl = 0 if l % m == 0 else m - l % m
    nc = 0 if c % m == 0 else m - c % m
    image_array=np.pad(image_array, ((0, nl), (0, nc)), mode='constant', constant_values=0)
    # data_h
    zigzag=[]
    for i in range(0,l,m):
        for j in range(0,c,m):
            R=image_array[i:i+m,j:j+m]
            zigzag.extend(I_zigzag(R))
    return zigzag

def pix_char(v):
    re=[]
    for i in v:
        re.append(chr(i))
    return re
def char_pix(lis):
    return [ord(i) for i in lis]

def decodage_LZ78(data):  
    dictionary = {0: ""}
    i = 1
    result = []
    for index, char in data:
        if index == 0:
            substr = char
        else:
            substr = dictionary[index] + char
        result.append(substr)
        dictionary[i] = substr
        i += 1
    return ''.join(result)

def retourner_image_zigzag(z,l,c):
    s=0
    q=np.zeros((max(l,c),max(l,c)))
    P = np.concatenate([np.arange(1, min(l, c)), [min(l, c)] * (abs(l - c) + 1), np.arange(min(l, c) - 1, 0, -1)])
    for i,j in zip(P,range(c-1,-l,-1)):
        lis=z[s:s+i]
        if(abs(j)%2 == (c-1)%2):
            r=list(reversed(lis))
        else:
            r=lis
        for k in range(max(l,c)-len(r)-abs(j)):
            r.append(0)
        e=np.diagflat(r,j)
        q=q+e
        s=s+i
    res=np.fliplr(q[:l,:c])
    return np.array(res,np.uint8)

def ordi(i):
    if(i!=''):
        return ord(i)
    return 0

def creer_pallete(image, num_colors):
    colors = list(image.getdata())
    color_counts = Counter(colors)
    sorted_colors = sorted(color_counts, key=color_counts.get, reverse=True)
    palette = sorted_colors[:num_colors]
    return palette

def closest_color_index(color, palette):
    distances = np.linalg.norm(np.array(palette) - np.array(color), axis=1)
    return np.argmin(distances)

def replace_colors_with_palette_indices(image, palette):

    image_array = np.array(image)
    l=image_array.shape[0]
    c=image_array.shape[1]
    imm=np.zeros((l,c))
    for i in range(l):
        for j in range(c):
            imm[i, j] = closest_color_index(image_array[i, j], palette)
    image_array = np.uint8(image_array)
    return imm


In [ ]:
import customtkinter
import tkinter as tk
from tkinter import filedialog
from tkinter import PhotoImage
import numpy as np
from PIL import Image
import itertools
from tkinter import ttk

message_label = None 
message_=None
message_label1=None
loading_text = None  
message_label2=None
emplacement_button = None

def on_closing():
    app.destroy()
    app.quit()
def EQM(image1,image2,l,c):
    #image sous forme array 
    l,c,_=image1.shape
    EQM=np.sum(image1-image2)/(l*c)
    return EQM
def damande_chem():
    global emplacement_button,message_label1
    message_label1 = myc.create_text(640, 400, text="Veuillez choisir l'emplacement de votre fichier :", font=("Bahnschrift", 20), fill="white")
    emplacement_button = customtkinter.CTkButton(master=app, text="parcourir", command=compression, width=10,bg_color='#121212',fg_color='black')
    _button_window = myc.create_window(950, 387, anchor="nw", window=emplacement_button)
    
def select_image():
    global emplacement_button, filepath,message_label,message_label1,photo

    filepath = filedialog.askopenfilename(filetypes=[("Image Files", "*.png;*.jpg;*.jpeg")])
    if filepath:
        afficher_intro()
        damande_chem()
        if message_label:
            myc.delete(message_label)

            
def show_compress_button():
    global compress_button, emplacement_fichier,message_label1
    emplacement_fichier = filedialog.asksaveasfile(defaultextension=".irm", filetypes=[("Fichiers IRM", "*.irm"), ("Tous les fichiers", "*.*")])
    emplacement_button.destroy() 
    if message_label1:
            myc.delete(message_label1)


def compression(): 
    global filepath,emplacement_button, emplacement_fichier,message_label,compress_button,emplacement_fichier,message_label1,message_label2,message_
        
    emplacement_fichier = filedialog.asksaveasfile(defaultextension=".irm", filetypes=[("Fichiers IRM", "*.irm"), ("Tous les fichiers", "*.*")])
    if(emplacement_fichier):
        emplacement_button.destroy() 
        myc.delete(message_label1)

        image = Image.open(filepath)
        mes = myc.create_text(200, 770, text="Traitement en cours... ", font=("Bahnschrift", 20), fill="white")
        myc.update()
        image_pal = image.convert('P', palette=Image.ADAPTIVE, colors=256)
        palette = image_pal.getpalette()
        data = np.array(image_pal)
        l, c = data.shape
        final_l = bytes()
        extension = b'irm'
        final_l += extension
        largeur_encodee = c.to_bytes(2, byteorder='big')
        hauteur_decodee = l.to_bytes(2, byteorder='big')
        final_l = final_l + largeur_encodee + hauteur_decodee
        indice = 2
        typelec_encodee = indice.to_bytes(1, byteorder='big')
        final_l += typelec_encodee
        liste_zigzag = I_zigzag(data)

        nouv_data = to_parts(data, 4)
        list_chaine = pix_char(nouv_data)
        list_chaine = ''.join(list_chaine)
        codage_lz78 = lz78c(list_chaine)
        m = max_codage_lz78(codage_lz78)
        bin_m = len(bin(m)[2:])
        q_coda = 1 if bin_m <= 8 else 2 if bin_m <= 16 else 3
        coda_b = q_coda.to_bytes(1, byteorder='big')
        final_l += coda_b
        for i in palette:
            final_l += i.to_bytes(1, byteorder='big')


        for i in range(0, len(codage_lz78)) :
                final_l += int(codage_lz78[i][0]).to_bytes(q_coda, byteorder='big')
                final_l += int(ordi(codage_lz78[i][1])).to_bytes(1, byteorder='big')

        file = emplacement_fichier.name
        with open(file, "wb") as f:
            f.write(final_l)
        taille_fich=len(final_l)
        taux=(1-(taille_fich/(l*c*3)))*100
        taux=round(taux,2)
        myc.delete(mes)
        image_c=np.zeros((l,c,3))
        for i in range(l):
            for j in range(c):
                image_c[i,j]=palette[data[i,j]*3:data[i,j]*3+3]
        image_c=np.array(image_c,np.uint8)
        image_original=np.array(image)
        EQM=np.sum(image_original-image_c)/(l*c)
        EQM=round(EQM,2)
        message_label = myc.create_text(250, 770, text="Votre image a été compressée ", font=("Bahnschrift", 20), fill="white")
        message_ = myc.create_text(1060, 730, text="EQM: "+str(EQM), font=("aBahnschrift", 15), fill="white")
        message_label2 = myc.create_text(1150, 770, text="Taux de compression: "+str(taux)+"%", font=("Bahnschrift", 15), fill="white")
    else:
        damande_chem()
def retourne_image():
    global message_label,message_,message_label2,emplacement_button,message_label1
    

    file_path = filedialog.askopenfilename(filetypes=[("IRM Files", "*.irm")])
    if file_path:
        with open(file_path, 'rb') as fichier:
            data = fichier.read()
        mes = myc.create_text(200, 770, text="Traitement en cours... ", font=("Bahnschrift", 20), fill="white")

        if message_label:
                myc.delete(message_label)
        if message_label2:
                myc.delete(message_label2)
        if message_label1:
                myc.delete(message_label1)
        if message_:
                myc.delete(message_)  
        if emplacement_button:
            emplacement_button.destroy()
        myc.update()



        c = int.from_bytes(data[3:5], byteorder='big')
        l = int.from_bytes(data[5:7], byteorder='big')
        ty = int.from_bytes(data[7:8], byteorder='big')
        N_o = int.from_bytes(data[8:9], byteorder='big')

        pallete = [i for i in data[9:777]]

        code_ind = [int.from_bytes(data[i:i+N_o], byteorder='big') for i in range(777, len(data), N_o+1)]
        code_char = [int.from_bytes(data[i+N_o:i+N_o+1], byteorder='big') for i in range(777, len(data), N_o+1)]
        code_lz = [(i, chr(j)) for i, j in zip(code_ind, code_char)]

        chaine = decodage_LZ78(code_lz)
        list_ = char_pix(chaine)

        nl = l if l % 4 == 0 else l + 4 - l % 4
        nc = c if c % 4 == 0 else c + 4 - c % 4

        block = [retourner_image_zigzag(list_[i:i+16], 4, 4) for i in range(0, len(list_), 16)]
        original_image = np.zeros((nl, nc))
        k = 0
        for i in range(0, nl, 4):
            for j in range(0, nc, 4):
                original_image[i:i+4, j:j+4] = block[k]
                k += 1

        original_image = np.array(original_image[:l, :c], np.uint8)

        image_ori = Image.fromarray(original_image)
        image_ori.putpalette(pallete)
        myc.delete(mes)
        image_ori.show()
def About_us():
    global photo
    myc.delete("all")
    myc.delete(photo)
    app.resizable(False, False)
    customtkinter.set_appearance_mode("dark")
    customtkinter.set_default_color_theme("dark-blue")
    myc.pack(fill="both", expand=True)
    myc.create_text(720, 200, text="Nom de l'application : Application", font=("Bahnschrift", 25), fill="black", anchor="center")
    myc.create_text(720, 250, text="Version : 1.0.0", font=("Bahnschrift", 25), fill="black", anchor="center")
    myc.create_text(720, 300, text="Description : description", font=("Bahnschrift", 25), fill="black", anchor="center")

def contact_us():
    global photo
    myc.delete("all")
    myc.delete(photo)
    app.resizable(False, False)
    customtkinter.set_appearance_mode("dark")
    customtkinter.set_default_color_theme("dark-blue")
    myc.pack(fill="both", expand=True)
    myc.create_text(720, 200, text="Email : TDM@email.com", font=("Bahnschrift", 25), fill="black", anchor="center")
    myc.create_text(720, 250, text="Contact : contact ", font=("Bahnschrift", 25), fill="black", anchor="center")
    


def afficher_intro():
    global photo
    myc.delete("all")
    myc.delete(photo)
    app.resizable(False, False)
    customtkinter.set_appearance_mode("dark")
    customtkinter.set_default_color_theme("dark-blue")
    
    menu=tk.Menu(app,font=("Arial", 12))
    app.config(menu=menu)

    file_menu = tk.Menu(menu, tearoff=0, font=("Arial", 14))  
    file_menu.configure(bg="#FFFFFF", fg="black") 
    file_menu.config(activebackground="#555555", activeforeground="white") 
    menu.add_cascade(label="File", menu=file_menu)

    file_menu.add_command(label="Acceuil",command=afficher_intro)
    file_menu.add_command(label="Open",command=select_image)
    file_menu.add_separator()
    file_menu.add_command(label="Exit", command=on_closing)
    
    help_menu = tk.Menu(menu, tearoff=0, font=("Arial", 14))
    help_menu.configure(bg="#FFFFFF", fg="black")
    help_menu.config(activebackground="#555555", activeforeground="white")
    menu.add_cascade(label="help", menu=help_menu)

    help_menu.add_command(label="about us",command=About_us)
    help_menu.add_command(label="contact us",command=contact_us)
    

    edit_menu = tk.Menu(menu, tearoff=0, font=("Arial", 14))
    edit_menu.configure(bg="#FFFFFF", fg="black")
    edit_menu.config(activebackground="#555555", activeforeground="white")
    menu.add_cascade(label="Edit", menu=edit_menu)

    edit_menu.add_command(label="Cut")
    edit_menu.add_command(label="Copy")
    edit_menu.add_command(label="Paste")
    
    photo = PhotoImage(master=app, file="back.png") 
    myc.pack(fill="both", expand=True)
    myc.create_image(0, 0, image=photo, anchor="nw")
    myc.create_text(680, 100, text="Logiciel de compression IRM", font=("Bahnschrift", 35), fill="white")
    
    button1 = customtkinter.CTkButton(master=app, text="Compression", command=select_image,bg_color='#080F17',fg_color='black')
    button2 = customtkinter.CTkButton(master=app, text="Décompression", command=retourne_image,bg_color='#101529',fg_color='black')


    butn_w1 = myc.create_window(350, 200, anchor="nw", window=button1)
    butn_w2 = myc.create_window(850, 200, anchor="nw", window=button2)


global  button1,button2,myc,app,photo
app = customtkinter.CTk()
app.geometry("1100x700")
app.title("Compression") 
app.resizable(False, False)
customtkinter.set_appearance_mode("white")
customtkinter.set_default_color_theme("dark-blue")


photo = PhotoImage(master=app, file="back0.png")  
myc = tk.Canvas(app, width=1100 , height=700)
myc.pack(fill="both", expand=True)
myc.create_image(0, 0, image=photo, anchor="nw")
myc.create_text(680, 100, text="intro", font=("Bahnschrift", 35), fill="white")

button1 = customtkinter.CTkButton(master=app, text="ouvrir", command=afficher_intro, bg_color='#FFFFFF', fg_color='black')
button2 = customtkinter.CTkButton(master=app, text="exit",command=on_closing,bg_color='#FFFFFF',fg_color='black')


butn_w1 = myc.create_window(680, 700, anchor="nw", window=button1)
butn_w2 = myc.create_window(900, 700, anchor="nw", window=button2)


app.protocol("WM_DELETE_WINDOW", on_closing)
app.mainloop()
